###   Background of code


We attempt to discover or mine information from the email dataset in different ways / visualization methods.

**For this Jupyter Notebook, all graphics are interactive. But because of the challenge in rendering interactive graphics in Github, the visualisations are not shown. It is recommended that you download and run the Jupyter Notebook at your own time :)**

###   Basic configuration steps

In [3]:
import os
os.getcwd()

'/home/kelvinhwee/PycharmProjects/enronFraudEmailAnalysis'

In [2]:
# - import basic python packages
import warnings
import tkinter  # to show plot in Pycharm

warnings.simplefilter(action='ignore', category=FutureWarning)

# - import packages for data manipulations
import numpy as np
import pandas as pd
from datetime import date, datetime
from collections import Counter
import random

# - import packages for visualisation
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = "browser"
# from pyvis.network import Network
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm

ModuleNotFoundError: No module named 'networkx'